In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("smartphoneData.xlsx")
df.head(4)

,name,brand,rating,price,release_year,os,display,camera,ram,battery,page_number,phone_position
0,Ulefone Armor 29 Pro Thermal,Ulefone,8.4,Coming soon,"Exp. release 2025, September 15",Android 15,"6.67"" 1080x2400 pixels",64MP 2160p,16GB RAM Dimensity 7400,21200mAh 120W,1,1
1,Ulefone Armor 29 Pro,Ulefone,8.5,Coming soon,"Exp. release 2025, September 15",Android 15,"6.67"" 1080x2400 pixels",64MP 2160p,16GB RAM Dimensity 7400,21200mAh 120W,1,2
2,Ulefone RugKing,Ulefone,6.0,Coming soon,"Exp. release 2025, September 15",Android 15,"5.99"" 720x1440 pixels",50MP 1080p,8GB RAM Unisoc T7255,9600mAh 18W,1,3
3,Cubot X100,Cubot,6.8,Coming soon,"2025, August",Android 15,"6.88"" 720x1640 pixels",64MP 1080p,8/12GB RAM Dimensity 7025,5100mAh 33W,1,4


In [3]:
df = df.drop(columns=['page_number', 'phone_position'])
df.columns

Index(['name', 'brand', 'rating', 'price', 'release_year', 'os', 'display',
       'camera', 'ram', 'battery'],
      dtype='object')

In [4]:
# Cleaning the 'price' column
def clean_price(price):
    # Handle 'Coming soon' and other non numeric values to 0
    if isinstance(price, str) and 'Coming soon' in price:
        return 0
    
    # Removing currency symbol (৳) and commas
    price = price.replace('৳', '').replace(',', '')
    
    # Splitting by space to handle cases with two prices (original and discounted)
    prices = price.split()
    
    # If two prices exist, take the second (discounted); otherwise, take the first
    selected_price = prices[-1] if len(prices) > 1 else prices[0]
    
    # Convert to float, handling any errors
    return float(selected_price)

# Apply the cleaning function to the price column
df['price'] = df['price'].apply(clean_price)
df['price']

0            0.0
1            0.0
2            0.0
3            0.0
4            0.0
          ...   
1195     12999.0
1196     19500.0
1197    100000.0
1198      8699.0
1199     27000.0
Name: price, Length: 1200, dtype: float64

In [5]:
# Cleaning the 'display' column
def clean_display(display):
    if isinstance(display, str):
        # Extract the screen size (e.g., "6.67" from "6.67\" 1080x2400 pixels")
        match = pd.Series(display).str.extract(r'(\d+\.\d+)\"').iloc[0, 0]
        if pd.notna(match):  # Check if match is not NaN
            try:
                return float(match)
            except ValueError:
                print(f"Failed to convert display: {display}, match: {match}")
                return np.nan
        return np.nan
    return np.nan

df['display'] = df['display'].apply(clean_display)
df['display']

0       6.67
1       6.67
2       5.99
3       6.88
4       4.70
        ... 
1195    6.60
1196    6.78
1197    6.81
1198    6.56
1199    6.67
Name: display, Length: 1200, dtype: float64

In [6]:
# Cleaning the 'camera' column
def clean_camera(camera):
    if isinstance(camera, str):
        if camera in ('No', 'N/A'):
            return 0
        match = pd.Series(camera).str.extract(r'(\d+)MP').iloc[0, 0]
        if pd.notna(match):
            try:
                return int(match)
            except ValueError:
                print(f"Failed to convert camera: {camera}, match: {match}")
                return 0
        return 0
    return 0

df['camera'] = df['camera'].apply(clean_camera)
df['camera']

0        64
1        64
2        50
3        64
4        48
       ... 
1195     52
1196     50
1197     50
1198      8
1199    200
Name: camera, Length: 1200, dtype: int64

In [7]:
# Cleaning the 'battery' column
def clean_battery(battery):
    if isinstance(battery, str):
        if battery in ('No', 'N/A'):
            return np.nan  # Change to 0 if preferred
        match = pd.Series(battery).str.extract(r'(\d+)mAh').iloc[0, 0]
        if pd.notna(match):
            try:
                return int(match)
            except ValueError:
                print(f"Failed to convert battery: {battery}, match: {match}")
                return np.nan
        return np.nan
    return np.nan

df['battery'] = df['battery'].apply(clean_battery)
df['battery']

0       21200.0
1       21200.0
2        9600.0
3        5100.0
4        4700.0
         ...   
1195     5000.0
1196     5000.0
1197     5200.0
1198     5000.0
1199     5100.0
Name: battery, Length: 1200, dtype: float64

In [8]:
df.head(3)

,name,brand,rating,price,release_year,os,display,camera,ram,battery
0,Ulefone Armor 29 Pro Thermal,Ulefone,8.4,0.0,"Exp. release 2025, September 15",Android 15,6.67,64,16GB RAM Dimensity 7400,21200.0
1,Ulefone Armor 29 Pro,Ulefone,8.5,0.0,"Exp. release 2025, September 15",Android 15,6.67,64,16GB RAM Dimensity 7400,21200.0
2,Ulefone RugKing,Ulefone,6.0,0.0,"Exp. release 2025, September 15",Android 15,5.99,50,8GB RAM Unisoc T7255,9600.0


In [9]:
df['os'][20:55].unique

<bound method Series.unique of 20      Android 15, up to 3 major Android upgrades
21                                      Android 15
22                                      Android 15
23                                      Android 15
24                             Android 15, HIOS 15
25                                      Android 15
26                            Android 15, XOS 15.1
27                           Android 15, HyperOS 2
28                           Android 14, MagicOS 8
29              Android 15, up to 4 major upgrades
30                                      Android 15
31                                      Android 15
32                                      Android 15
33                                      Android 15
34      Android 15, up to 2 major upgrades, XOS 15
35                          Android 15, Magic OS 9
36                                      Android 15
37       Android 15, up to 1 major Android upgrade
38                                      Android 14


In [10]:
# Cleaning the 'os' column
def clean_os(os):
    if isinstance(os, str):
        base_os = os.split(',')[0].strip()
        valid_os = ('Android', 'HarmonyOS', 'KaiOS', 'EMUI', 'iOS', 'Tizen', 'ColorOS', 'OxygenOS')
        if base_os.startswith(valid_os):
            return base_os
        return np.nan
    return np.nan

df['os'] = df['os'].apply(clean_os)

In [11]:
df['os'].info

<bound method Series.info of 0       Android 15
1       Android 15
2       Android 15
3       Android 15
4       Android 15
           ...    
1195    Android 12
1196    Android 13
1197           NaN
1198    Android 13
1199    Android 13
Name: os, Length: 1200, dtype: object>

In [12]:
df.head(4)

,name,brand,rating,price,release_year,os,display,camera,ram,battery
0,Ulefone Armor 29 Pro Thermal,Ulefone,8.4,0.0,"Exp. release 2025, September 15",Android 15,6.67,64,16GB RAM Dimensity 7400,21200.0
1,Ulefone Armor 29 Pro,Ulefone,8.5,0.0,"Exp. release 2025, September 15",Android 15,6.67,64,16GB RAM Dimensity 7400,21200.0
2,Ulefone RugKing,Ulefone,6.0,0.0,"Exp. release 2025, September 15",Android 15,5.99,50,8GB RAM Unisoc T7255,9600.0
3,Cubot X100,Cubot,6.8,0.0,"2025, August",Android 15,6.88,64,8/12GB RAM Dimensity 7025,5100.0


In [14]:
import re
def clean_ram_simple(value):
    # Handle missing values
    if pd.isna(value) or value == '' or value == 0:
        return 0

    text = str(value)
    
    # Find all numbers in the text
    numbers = []
    for match in re.finditer(r'\d+', text):
        numbers.append(int(match.group()))
    
    if not numbers:
        return 0
    
    # Check if MB is mentioned
    if 'MB' in text or 'mb' in text:
        # Take first number, convert MB to GB
        return int(numbers[0] / 1024)
    else:
        # Assume GB, take minimum number for ranges
        return min(numbers)

df['ram'] = df['ram'].apply(clean_ram_simple)
df['ram'].info

<bound method Series.info of 0       16
1       16
2        8
3        8
4        6
        ..
1195     8
1196     8
1197     2
1198     3
1199     2
Name: ram, Length: 1200, dtype: int64>

In [15]:
df.head(4)

,name,brand,rating,price,release_year,os,display,camera,ram,battery
0,Ulefone Armor 29 Pro Thermal,Ulefone,8.4,0.0,"Exp. release 2025, September 15",Android 15,6.67,64,16,21200.0
1,Ulefone Armor 29 Pro,Ulefone,8.5,0.0,"Exp. release 2025, September 15",Android 15,6.67,64,16,21200.0
2,Ulefone RugKing,Ulefone,6.0,0.0,"Exp. release 2025, September 15",Android 15,5.99,50,8,9600.0
3,Cubot X100,Cubot,6.8,0.0,"2025, August",Android 15,6.88,64,8,5100.0


In [16]:
rename_columns = {
    'price': 'price_bdt',
    'display': 'display_inches',
    'camera': 'camera_megapixel',
    'ram': 'ram_gb',
    'battery': 'battery_mah'
}

In [17]:
df.rename(columns=rename_columns, inplace=True)

In [18]:
df.columns.tolist()

['name',
 'brand',
 'rating',
 'price_bdt',
 'release_year',
 'os',
 'display_inches',
 'camera_megapixel',
 'ram_gb',
 'battery_mah']

In [19]:
df.head(4)

,name,brand,rating,price_bdt,release_year,os,display_inches,camera_megapixel,ram_gb,battery_mah
0,Ulefone Armor 29 Pro Thermal,Ulefone,8.4,0.0,"Exp. release 2025, September 15",Android 15,6.67,64,16,21200.0
1,Ulefone Armor 29 Pro,Ulefone,8.5,0.0,"Exp. release 2025, September 15",Android 15,6.67,64,16,21200.0
2,Ulefone RugKing,Ulefone,6.0,0.0,"Exp. release 2025, September 15",Android 15,5.99,50,8,9600.0
3,Cubot X100,Cubot,6.8,0.0,"2025, August",Android 15,6.88,64,8,5100.0


In [20]:
df.to_csv('Preprocessed_Smartphone_DATA.csv', index=False)
df.to_excel('Preprocessed_SmartphoneDATA.xlsx', index=False)